In [2]:
import polars as pl

base_dir = '../data/mitsui-commodity-prediction-challenge'
train_dir = '/train.csv'
train_labels_dir = '/train_labels.csv'
target_pairs_dir = '/target_pairs.csv'

In [14]:
train_df = pl.read_csv(base_dir + train_dir)
train_df

date_id,LME_AH_Close,LME_CA_Close,LME_PB_Close,LME_ZS_Close,JPX_Gold_Mini_Futures_Open,JPX_Gold_Rolling-Spot_Futures_Open,JPX_Gold_Standard_Futures_Open,JPX_Platinum_Mini_Futures_Open,JPX_Platinum_Standard_Futures_Open,JPX_RSS3_Rubber_Futures_Open,JPX_Gold_Mini_Futures_High,JPX_Gold_Rolling-Spot_Futures_High,JPX_Gold_Standard_Futures_High,JPX_Platinum_Mini_Futures_High,JPX_Platinum_Standard_Futures_High,JPX_RSS3_Rubber_Futures_High,JPX_Gold_Mini_Futures_Low,JPX_Gold_Rolling-Spot_Futures_Low,JPX_Gold_Standard_Futures_Low,JPX_Platinum_Mini_Futures_Low,JPX_Platinum_Standard_Futures_Low,JPX_RSS3_Rubber_Futures_Low,JPX_Gold_Mini_Futures_Close,JPX_Gold_Rolling-Spot_Futures_Close,JPX_Gold_Standard_Futures_Close,JPX_Platinum_Mini_Futures_Close,JPX_Platinum_Standard_Futures_Close,JPX_RSS3_Rubber_Futures_Close,JPX_Gold_Mini_Futures_Volume,JPX_Gold_Rolling-Spot_Futures_Volume,JPX_Gold_Standard_Futures_Volume,JPX_Platinum_Mini_Futures_Volume,JPX_Platinum_Standard_Futures_Volume,JPX_RSS3_Rubber_Futures_Volume,JPX_Gold_Mini_Futures_settlement_price,JPX_Gold_Rolling-Spot_Futures_settlement_price,…,FX_AUDUSD,FX_CADJPY,FX_CHFJPY,FX_EURAUD,FX_EURGBP,FX_EURJPY,FX_EURUSD,FX_GBPAUD,FX_GBPJPY,FX_GBPUSD,FX_NZDJPY,FX_NZDUSD,FX_USDCHF,FX_USDJPY,FX_ZARJPY,FX_ZARUSD,FX_NOKUSD,FX_NOKEUR,FX_CADUSD,FX_AUDNZD,FX_EURCHF,FX_EURCAD,FX_AUDCAD,FX_GBPCHF,FX_EURNZD,FX_AUDCHF,FX_GBPNZD,FX_GBPCAD,FX_CADCHF,FX_NZDCAD,FX_NZDCHF,FX_ZAREUR,FX_NOKGBP,FX_NOKCHF,FX_ZARCHF,FX_NOKJPY,FX_ZARGBP
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,2264.5,7205.0,2570.0,3349.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.783393,89.764583,115.545884,1.540368,0.887622,135.44991,1.206713,1.735387,152.598635,1.35949,79.721293,0.710231,0.97145,112.247002,9.028141,0.080431,0.123146,0.102051,0.799706,1.103011,1.172261,1.508946,0.979601,1.320676,1.699043,0.761027,1.914152,1.699987,0.776874,0.888115,0.689954,0.066653,0.090582,0.11963,0.078135,13.82274,0.059163
1,2228.0,7147.0,2579.0,3327.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,0.782779,89.731507,115.23516,1.534906,0.889248,135.272384,1.201492,1.726072,152.119947,1.351132,79.815128,0.70892,0.977019,112.586998,9.111194,0.080926,0.123355,0.102668,0.796997,1.104186,1.173881,1.507524,0.98216,1.320083,1.694821,0.76479,1.905904,1.695279,0.778682,0.889488,0.692628,0.067354,0.091297,0.12052,0.079066,13.888146,0.059895
2,2250.0,7188.5,2587.0,3362.0,4684.0,4691.0,4684.0,3363.0,3367.0,207.0,4735.0,4746.0,4735.0,3443.0,3449.0,207.7,4679.0,4688.0,4679.0,3362.0,3362.0,206.4,4727.0,4739.0,4730.0,3426.0,3427.0,206.9,2681.0,37908.0,30656.0,624.0,13713.0,4128.0,4730.0,4735.0,…,0.786472,90.306741,115.750255,1.535084,0.89083,136.176212,1.2073,1.723206,152.864365,1.355253,80.734616,0.71577,0.97446,112.793999,9.177484,0.081365,0.123975,0.102688,0.800634,1.098776,1.176466,1.50793,0.982311,1.32064,1.686714,0.766385,1.893418,1.692724,0.780186,0.894004,0.69749,0.067394,0.091478,0.120809,0.079287,13.983675,0.060037
3,2202.5,7121.0,2540.0,3354.0,4728.0,4737.0,4729.0,3430.0,3426.0,207.0,4795.0,4797.0,4789.0,3493.0,3494.0,207.6,4727.0,4734.0,4726.0,3430.0,3425.0,204.0,4778.0,4788.0,4780.0,3486.0,3486.0,204.8,3523.0,30791.0,35267.0,525.0,17629.0,4586.0,4780.0,4787.0,…,0.787461,91.079905,115.976719,1.527972,0.886809,135.945666,1.203219,1.723001,153.297609,1.356796,81.010025,0.716998,0.974204,112.985001,9.195264,0.081385,0.124225,0.103244,0.806124,1.098275,1.172181,1.492598,0.976849,1.321796,1.678134,0.767148,1.892329,1.683111,0.785329,0.889439,0.698502,0.067639,0.091558,0.121021,0.079285,1

In [5]:
from itertools import combinations

def create_features(df):
    feature_cols = [col for col in df.columns if col != 'date_id']
    lag_days = [1, 2, 3, 5, 10, 20]
    windows = [3, 5, 10, 15, 20]
    change_days = [1, 3, 5, 7, 14, 21]
    exprs = []
    
    lag_exprs = [
        pl.col(col).shift(day).alias(f'{col}_lag_{day}')
        for col in feature_cols
        for day in lag_days
    ]
    
    rolling_exprs = [
        expr.alias(f'{col}_rolling_{stat_name}_{window}')
        for col in feature_cols
        for window in windows
        for stat_name, expr in [
            ('mean', pl.col(col).rolling_mean(window_size=window)),
            ('std', pl.col(col).rolling_std(window_size=window)),
            ('max', pl.col(col).rolling_max(window_size=window)),
            ('min', pl.col(col).rolling_min(window_size=window)),
        ]
    ]
    
    change_exprs = [
        pl.col(col).pct_change(n=day).alias(f'change_{col}_{day}')
        for col in feature_cols
        for day in change_days
    ]
    
    exprs.extend(lag_exprs)
    exprs.extend(rolling_exprs)
    exprs.extend(change_exprs)

    for col1, col2 in combinations(feature_cols, 2):
        exprs.append(
            (pl.col(col1) - pl.col(col2)).alias(f'spread_{col1}_vs_{col2}')
        )
        exprs.append(
            pl.corr(col1, col2).alias(f'corr_{col1}_vs_{col2}')
        )
        
    df_features = df.with_columns(exprs).fill_null(strategy="forward")
    
    return df_features

In [ ]:
train_df_features = create_features(train_df)
train_df_features


KeyboardInterrupt



In [ ]:
target_pairs = pl.read_csv(base_dir + target_pairs_dir)
target_pairs
target_1 = target_pairs.filter(pl.col('target') == 'target_1')\
                       .select('pair')\
                       .item()
                        
target_1
com1, com2 = target_1.split(' - ')

'US_Stock_VT_adj_close'

In [11]:
train_labels = pl.read_csv(base_dir + train_labels_dir)
train_labels
train_label_val = train_labels.filter(pl.col('date_id') == 0)\
                              .select('target_1')\
                              .item()
train_label_val

-0.0028513218171679

In [ ]:
train_df_filtered = train_df.select(['date_id', com1, com2])\
                            .filter(pl.col('date_id') == 0)
train_df_filtered

val1 = train_df_filtered.select(com1)\
                        .item()
val2 = train_df_filtered.select(com2)\
                        .item()
                        
val1, val2



(2570.0, 63.9271)

In [ ]:
train_labels = pl.read_csv(base_dir + train_labels_dir)
train_labels

In [ ]:
target_pairs = pl.read_csv(base_dir + target_pairs_dir)
target_pairs

In [ ]:
import polars as pl
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

required_cols = ["date_id", "LME_PB_Close", "US_Stock_VT_adj_close"]
train_df = pl.read_csv('../data/mitsui-commodity-prediction-challenge/train.csv', columns=required_cols)
train_labels_df = pl.read_csv('../data/mitsui-commodity-prediction-challenge/train_labels.csv', columns=["date_id", "target_1"])
TEST_START_DATE = 1827


target_name = 'target_1'
lag = 1
asset_a = 'LME_PB_Close'
asset_b = 'US_Stock_VT_adj_close'
feature_name = 'engineered_feature_for_target_1'

train_data_actual = train_df.filter(pl.col('date_id') < TEST_START_DATE)
test_data_actual = train_df.filter(pl.col('date_id') >= TEST_START_DATE)
train_labels_actual = train_labels_df.filter(pl.col('date_id') < TEST_START_DATE)

log_return_a = (pl.col(asset_a).shift(1) / pl.col(asset_a).shift(1 + lag)).log()
log_return_b = (pl.col(asset_b).shift(1) / pl.col(asset_b).shift(1 + lag)).log()
engineered_feature_expr = (log_return_a - log_return_b).alias(feature_name)

train_with_features = train_data_actual.with_columns(engineered_feature_expr)
training_data_final = train_with_features.join(train_labels_actual, on='date_id')
training_data_final = training_data_final.drop_nulls(subset=[feature_name, target_name])

X_train = training_data_final.select(feature_name).to_numpy()
y_train = training_data_final.select(target_name).to_numpy().ravel()

model = Ridge(alpha=1.0)
model.fit(X_train, y_train)

learned_coefficient = model.coef_[0]
learned_intercept = model.intercept_
print(f"✅ Model trained! Formula: Prediction = ({learned_coefficient:.4f} * Feature) + {learned_intercept:.4f}")
print("-" * 50)

combined_for_prediction = pl.concat([
    train_data_actual.tail(lag + 1),
    test_data_actual
])

prediction_data_with_feature = combined_for_prediction.with_columns(engineered_feature_expr)
test_features_df = prediction_data_with_feature.filter(pl.col('date_id') >= TEST_START_DATE)

X_pred = test_features_df.select(
    pl.col(feature_name).fill_null(strategy='forward')
).to_numpy()

all_predictions = model.predict(X_pred)

results_df = test_features_df.select("date_id").with_columns(
    pl.Series(name="prediction", values=all_predictions)
)

comparison_df = results_df.join(train_labels_df, on='date_id', how='left')
comparison_df = comparison_df.drop_nulls()

print(comparison_df.head())

mse = mean_squared_error(comparison_df.select('target_1'), comparison_df.select('prediction'))
print(f"\n🎯 Mean Squared Error on Test Set: {mse:.8f}")

🎯 Preparing to predict for: target_1
Definition: Return(LME_PB_Close) - Return(US_Stock_VT_adj_close) with lag=1
--------------------------------------------------
🏋️  Training the Ridge model on data before date_id 1827...
✅ Model trained! Formula: Prediction = (-0.0137 * Feature) + -0.0005
--------------------------------------------------
📈 Making predictions for all dates from 1827 onwards...

✨ Comparison of Predictions vs. Actuals (First 5 Rows):
shape: (5, 3)
┌─────────┬────────────┬───────────┐
│ date_id ┆ prediction ┆ target_1  │
│ ---     ┆ ---        ┆ ---       │
│ i64     ┆ f64        ┆ f64       │
╞═════════╪════════════╪═══════════╡
│ 1828    ┆ -0.000409  ┆ -0.004592 │
│ 1829    ┆ -0.000409  ┆ -0.014539 │
│ 1830    ┆ -0.000409  ┆ -0.005226 │
│ 1831    ┆ -0.000485  ┆ 0.016613  │
│ 1832    ┆ -0.000349  ┆ -0.013144 │
└─────────┴────────────┴───────────┘

🎯 Mean Squared Error on Test Set: 0.00033464
